In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD, AdamW
from tensorflow.keras.datasets import cifar10
import numpy as np
import time


In [2]:
# Load CIFAR-10 dataset
(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()

# Normalize images to range [0, 1]
x_train_full = x_train_full.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 225s 1us/step


In [3]:
# Split training data into Train and Validation sets
# 50,000 images total, 5,000 used for validation
validation_size = 5000
x_valid, x_train = x_train_full[:validation_size], x_train_full[validation_size:]
y_valid, y_train = y_train_full[:validation_size], y_train_full[validation_size:]


In [4]:
# Convert labels to One-Hot Encoding
# CategoricalCrossentropy requires one-hot encoded labels
# (SparseCategoricalCrossentropy could be used instead to avoid this step)
y_train_cat = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_valid_cat = tf.keras.utils.to_categorical(y_valid, num_classes=10)
y_test_cat = tf.keras.utils.to_categorical(y_test, num_classes=10)


In [22]:
# Optional: Use a smaller subset of training data (resource-limited setting)
x_train = x_train[:10000]
y_train = y_train[:10000]
y_train_cat = y_train_cat[:10000]


In [5]:
# Base Model Definition

def create_base_model():
    model = Sequential([
        # First convolutional layer
        Conv2D(64, (4, 4), activation='relu',
               input_shape=(32, 32, 3), padding='same'),
        
        # First max pooling layer
        MaxPooling2D((2, 2)),
        
        # Second convolutional layer
        Conv2D(32, (4, 4), activation='relu', padding='same'),
        
        # Second max pooling layer
        MaxPooling2D((2, 2)),
        
        # Flatten for dense layers
        Flatten(),
        
        # Fully connected layer
        Dense(256, activation='relu'),
        
        # Output layer (10 classes)
        Dense(10, activation='softmax')
    ])
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


In [6]:
# Create and summarize the base model
base_model = create_base_model()
base_model.summary()


d:\Uni\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 64)     │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 32)     │        32,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 563,050 (2.15 MB)

 Trainable params: 563,050 (2.15 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Train the base model (for final reporting)
history_base = base_model.fit(
    x_train, y_train_cat,
    epochs=50,
    batch_size=32,
    validation_data=(x_valid, y_valid_cat),
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)


Epoch 1/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 27s 18ms/step - accuracy: 0.4804 - loss: 1.4466 - val_accuracy: 0.5944 - val_loss: 1.1689
Epoch 2/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 25s 18ms/step - accuracy: 0.6195 - loss: 1.0678 - val_accuracy: 0.6348 - val_loss: 1.0358
Epoch 3/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 28s 20ms/step - accuracy: 0.6838 - loss: 0.9014 - val_accuracy: 0.6664 - val_loss: 0.9613
Epoch 4/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 30s 21ms/step - accuracy: 0.7273 - loss: 0.7706 - val_accuracy: 0.6954 - val_loss: 0.9091
Epoch 5/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 26s 19ms/step - accuracy: 0.7683 - loss: 0.6561 - val_accuracy: 0.6960 - val_loss: 0.9210
Epoch 6/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 24s 17ms/step - accuracy: 0.8018 - loss: 0.5556 - val_accuracy: 0.7020 - val_loss: 0.9416
Epoch 7/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 27s 19ms/step - accuracy: 0.8397 - loss: 0.4543 - val_accuracy: 0.6926 - val_loss: 1.0258
Epoch 8/50
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 28s 20ms/step - accuracy: 0.8697 -

In [7]:
# Model (a): Reduced Channels + Dropout

def create_model_a():
    model = Sequential([
        # First convolutional layer: reduced filters (64 → 32)
        Conv2D(32, (4, 4), activation='relu',
               input_shape=(32, 32, 3), padding='same'),
        MaxPooling2D((2, 2)),
        
        # Second convolutional layer
        Conv2D(32, (4, 4), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        
        Flatten(),
        
        # Fully connected layer
        Dense(256, activation='relu'),
        
        # Dropout with 30% probability
        Dropout(0.3),
        
        # Output layer
        Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


In [8]:
# Training and Evaluation Framework

def run_experiment(
    model_creator,
    name,
    x_train_data, y_train_data,
    x_valid_data, y_valid_data,
    x_test_data, y_test_data,
    epochs=50,
    batch_size=32,
    runs=2
):
    accuracies = []
    training_times = []
    
    print(f"--- Starting experiment: {name} ---")
    
    for i in range(runs):
        tf.random.set_seed(42 + i)  # Reproducibility
        model = model_creator()
        
        start_time = time.time()
        
        # Early stopping based on validation loss
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
        
        history = model.fit(
            x_train_data, y_train_data,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(x_valid_data, y_valid_data),
            callbacks=[early_stopping],
            verbose=0
        )
        
        training_time = time.time() - start_time
        training_times.append(training_time)
        
        # Final evaluation on test set
        _, accuracy = model.evaluate(x_test_data, y_test_data, verbose=0)
        accuracies.append(accuracy)
        
        print(
            f"Run {i+1}: Test Accuracy = {accuracy:.4f}, "
            f"Training Time = {training_time:.2f} seconds"
        )
    
    avg_accuracy = np.mean(accuracies)
    avg_time = np.mean(training_times)
    
    print(f"\nFinal Results for {name}:")
    print(f"Average Test Accuracy (2 runs): {avg_accuracy:.4f}")
    print(f"Average Training Time (2 runs): {avg_time:.2f} seconds")
    
    return history, avg_accuracy, avg_time


In [12]:
# Run experiments

history_base, acc_base, time_base = run_experiment(
    create_base_model,
    "Base Model",
    x_train, y_train_cat,
    x_valid, y_valid_cat,
    x_test, y_test_cat
)

history_a, acc_a, time_a = run_experiment(
    create_model_a,
    "Model A",
    x_train, y_train_cat,
    x_valid, y_valid_cat,
    x_test, y_test_cat
)


--- Starting experiment: Base Model ---
Run 1: Test Accuracy = 0.6531, Training Time = 265.35 seconds
Run 2: Test Accuracy = 0.6655, Training Time = 259.02 seconds

Final Results for Base Model:
Average Test Accuracy (2 runs): 0.6593
Average Training Time (2 runs): 262.19 seconds
--- Starting experiment: Model A ---
Run 1: Test Accuracy = 0.6903, Training Time = 233.20 seconds
Run 2: Test Accuracy = 0.6763, Training Time = 194.32 seconds

Final Results for Model A:
Average Test Accuracy (2 runs): 0.6833
Average Training Time (2 runs): 213.76 seconds


In [14]:
def create_model_b():
    model = Sequential([
        # لایه 1
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), padding='same'),
        MaxPooling2D((2, 2)),
        
        # لایه 2
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        
        # لایه 3 (جدید) - بدون Max Pooling
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        
        # لایه 4 (جدید) - بدون Max Pooling
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        
        Flatten(),
        
        Dense(256, activation='relu'),
        
        Dense(10, activation='softmax')
    ])
    
    # استفاده از همان هایپرپارامترهای پایه
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [15]:

model_b = create_model_b()
model_b.summary()
history_b, acc_b, time_b = run_experiment(create_model_b, "Model B", x_train, y_train_cat, x_valid, y_valid_cat, x_test, y_test_cat)

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 16, 16, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 8, 8, 32)       │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 8, 8, 32)       │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 555,754 (2.12 MB)

 Trainable params: 555,754 (2.12 MB)

 Non-trainable params: 0 (0.00 B)

--- Starting experiment: Model B ---
Run 1: Test Accuracy = 0.6357, Training Time = 143.63 seconds
Run 2: Test Accuracy = 0.6911, Training Time = 149.63 seconds

Final Results for Model B:
Average Test Accuracy (2 runs): 0.6634
Average Training Time (2 runs): 146.63 seconds


In [18]:
def create_model_p():
    model = Sequential([
        # لایه پیچشی اول
        Conv2D(64, (4, 4), padding='same', input_shape=(32, 32, 3)),
        BatchNormalization(), # <-- BN
        tf.keras.layers.Activation('relu'), # <-- ReLU
        MaxPooling2D((2, 2)),
        
        # لایه پیچشی دوم
        Conv2D(32, (4, 4), padding='same'),
        BatchNormalization(), # <-- BN
        tf.keras.layers.Activation('relu'), # <-- ReLU
        MaxPooling2D((2, 2)),
        
        # صاف کردن (Flatten)
        Flatten(),
        
        # لایه کاملاً متصل اول
        Dense(256),
        BatchNormalization(), # <-- BN
        tf.keras.layers.Activation('relu'), # <-- ReLU
        
        # لایه کاملاً متصل خروجی
        Dense(10, activation='softmax')
    ])
    
    # استفاده از همان هایپرپارامترهای پایه
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [19]:
model_p = create_model_p()
model_p.summary()
history_p, acc_p, time_p = run_experiment(create_model_p, "Model P (Batch Norm)", x_train, y_train_cat, x_valid, y_valid_cat, x_test, y_test_cat)

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_30 (Conv2D)              │ (None, 32, 32, 64)     │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 16, 16, 32)     │        32,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 16, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_12 (Flatten)            │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 564,458 (2.15 MB)

 Trainable params: 563,754 (2.15 MB)

 Non-trainable params: 704 (2.75 KB)

--- Starting experiment: Model P (Batch Norm) ---
Run 1: Test Accuracy = 0.6586, Training Time = 473.07 seconds
Run 2: Test Accuracy = 0.6342, Training Time = 395.60 seconds

Final Results for Model P (Batch Norm):
Average Test Accuracy (2 runs): 0.6464
Average Training Time (2 runs): 434.34 seconds


In [ ]:
def create_model_t(lr):
    model = Sequential([
        # همان معماری پایه
        Conv2D(64, (4, 4), activation='relu', input_shape=(32, 32, 3), padding='same'),
        MaxPooling2D((2, 2)),
        Conv2D(32, (4, 4), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dense(10, activation='softmax')
    ])
    
    # کامپایل با نرخ یادگیری متفاوت
    model.compile(optimizer=Adam(learning_rate=lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [ ]:

history_t_1, acc_t_1, time_t_1 = run_experiment(lambda: create_model_t(0.1), "Model T (LR=0.1)", x_train, y_train_cat, x_valid, y_valid_cat, x_test, y_test_cat)
history_t_3, acc_t_3, time_t_3 = run_experiment(lambda: create_model_t(0.001), "Model T (LR=0.001) [Base]", x_train, y_train_cat, x_valid, y_valid_cat, x_test, y_test_cat)
history_t_4, acc_t_4, time_t_4 = run_experiment(lambda: create_model_t(0.0001), "Model T (LR=0.0001)", x_train, y_train_cat, x_valid, y_valid_cat, x_test, y_test_cat)

--- Starting experiment: Model T (LR=0.1) ---
